In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [246]:
dataset = pd.read_csv('/Users/anshulpattoo/Desktop/CISC 251/Project/Python scripts/merged.csv', index_col=0)
print(dataset)

                          people_NNS  america_NP   make_VB  president_NN  \
1992clinton920416           0.221278    0.287723  0.354953      0.168776   
1992clinton920422           0.805724    0.125871  0.251418      0.174340   
1992clinton920716           0.417194    0.668999  0.290108      0.190195   
1992clinton920814           0.069395    0.084924  0.318052      0.317582   
1992clinton921002           0.396249    0.000000  0.000000      0.644756   
...                              ...         ...       ...           ...   
2012romney120925            0.384952    0.147214  0.176436      0.195744   
2012romney121003debate1r    0.573114    0.211366  0.302245      0.303396   
2012romney121016debate2r    0.620353    0.276066  0.531707      0.442440   
2012romney121022debate3     0.239600    0.293216  0.503856      0.372672   
2012romney121107            0.287569    0.586529  0.000000      0.292451   

                          country_NN  going_VBG    us_PPO  american_JJ  \
1992clinton92

In [247]:
print(dataset.iloc[:, :818].values)

[[0.22127783 0.28772333 0.35495283 ... 0.         0.10214049 1.        ]
 [0.8057242  0.12587122 0.25141774 ... 0.         0.         1.        ]
 [0.41719428 0.66899876 0.29010759 ... 0.         0.212496   1.        ]
 ...
 [0.6203527  0.27606581 0.53170712 ... 0.         0.         0.        ]
 [0.2395996  0.29321599 0.50385573 ... 0.         0.15613857 0.        ]
 [0.28756866 0.58652928 0.         ... 0.         0.         0.        ]]


In [249]:
# split into training and testing sets
X, y = dataset.iloc[:, :817].values, dataset.iloc[:, 817].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3,
    stratify=y, random_state=0
)
# standardize the features (also known as z-score normalization)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

X_train_std = X_train
X_test_std = X_test

print(len(X_train_std))
print(len(X_test_std))

301
130


In [226]:
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

In [227]:
# calculate cumulative sum of explained variances
tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [228]:
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i]) for i in range(len(eigen_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eigen_pairs.sort(key=lambda k: k[0], reverse=True)

In [237]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis], eigen_pairs[1][1][:, np.newaxis]))

817


In [240]:
X_train_std[0].dot(w)


array([ 0.12486962+0.j, -0.24739107+0.j])

In [243]:
X_train_pca = X_train_std.dot(w)
print(len(y_train))

301


In [232]:
import matplotlib.pyplot as plt
colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']
#for l, c, m in zip(np.unique(y_train), colors, markers):
#     plt.scatter(X_train_pca[y_train==l, 0], 
#                 X_train_pca[y_train==l, 1], 
#                 c=c, label=l, marker=m) 
# print(X_train_pca[y_train==1, 0])
# print(X_train_pca[y_train==1, 1])

dataset = [[], []]

for i in range(len(X_train_pca[y_train==1, 0])):
    dataset[0].append(X_train_pca[y_train==1, 0][i].real)
    dataset[1].append(X_train_pca[y_train==1, 1][i].real)

#print(dataset)
# plt.xlabel('PC 1')
# plt.ylabel('PC 2')
# plt.legend(loc='lower left')
# plt.show()

In [233]:
from pandas import DataFrame
#np.array(dataset).reshape(404, )

df = pd.DataFrame({'PC1': dataset[0],
                   'PC2': dataset[1]})
print(df)


          PC1       PC2
0   -1.522930  0.017321
1   -2.008482 -1.311938
2   -1.590624 -0.734623
3   -2.002195 -1.423006
4    0.532929  0.343599
..        ...       ...
197 -0.890827 -0.815827
198 -1.791007 -0.697696
199 -1.424033  0.837935
200 -3.078324  0.677081
201 -2.537797  0.705714

[202 rows x 2 columns]


In [234]:
from sklearn.metrics import silhouette_score

EM = GaussianMixture(n_components = 2) 
EM.fit(df)

cluster = EM.predict(df)

print("SILHOUETTE: ", silhouette_score(df, cluster))
df['cluster'] = cluster


SILHOUETTE:  0.38291630137794586


In [236]:
import pandas as pd
import numpy as np
from plotnine import *
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_score

print((cluster == 0).sum())
print((cluster == 1).sum())

78
124


In [ ]:
(ggplot(df, aes(x = "PC1", y = "PC2", color = "cluster")) + geom_point())